# Local Volatility

Reference:
[*The Volatility Smile and Its Implied Tree*](https://emanuelderman.com/the-volatility-smile-and-its-implied-tree/) - Emanuel Derman & Iraj Kani

Normal CRR Tree

In [1]:
import numpy as np
from scipy.stats import norm
import math

In [34]:
def binomial_price(sigma, K, r, S0, T, m, is_call=True, is_euro=True, q=0):
    """
    CRR option pricing
    :param sigma: volatility of the underlying (decimal)
    :param K: strike price
    :param r: risk-free rate (decimal)
    :param S0: initial asset price
    :param T: time until expiration (years)
    :param m: number of periods after initial
    :param is_call: True for call, False for put
    :param is_euro: True for European, False for American
    :param q: dividend yield (decimal)

    :return: price of option
    """
    dt = T / m
    u = np.exp(sigma * np.sqrt(dt))  # up movement size
    d = 1 / u  # down movement size
    p = (np.exp((r - q) * dt) - d) / (u - d)  # risk-neutral up probability

    price_tree = np.zeros((m + 1, m + 1))
    option_values = np.zeros((m + 1, m + 1))

    for j in range(m + 1):
        for i in range(j + 1):
            price_tree[i, j] = S0 * (u ** (j - i)) * (d ** i)

    if is_call:
        option_values[:, m] = np.maximum(np.zeros(m + 1), price_tree[:, m] - K)
    else:
        option_values[:, m] = np.maximum(np.zeros(m + 1), K - price_tree[:, m])

    for j in range(m - 1, -1, -1):
        for i in range(j + 1):
            cont = np.exp(-r * dt) * (p * option_values[i, j + 1] + (1 - p) * option_values[i + 1, j + 1])
            if is_euro or is_call:
                option_values[i, j] = cont
            else:
                option_values[i, j] = max(K - price_tree[i, j], cont)

    return option_values[0, 0]

In [2]:
T = 5
dt = 1  
m = int(T/dt)
S0 = 100
r = 0.03
sigma = 0.10
q = 0

u = np.exp(sigma * np.sqrt(dt))  # up movement size
d = 1 / u  # down movement size
p = (np.exp((r - q) * dt) - d) / (u - d)  # risk-neutral up probability

price_tree = np.zeros((m + 1, m + 1))
option_values = np.zeros((m + 1, m + 1))

for j in range(m + 1):
    for i in range(j + 1):
        price_tree[i, j] = S0 * (u ** (j - i)) * (d ** i)

In [3]:
price_tree

array([[100.        , 110.51709181, 122.14027582, 134.98588076,
        149.18246976, 164.87212707],
       [  0.        ,  90.4837418 , 100.        , 110.51709181,
        122.14027582, 134.98588076],
       [  0.        ,   0.        ,  81.87307531,  90.4837418 ,
        100.        , 110.51709181],
       [  0.        ,   0.        ,   0.        ,  74.08182207,
         81.87307531,  90.4837418 ],
       [  0.        ,   0.        ,   0.        ,   0.        ,
         67.0320046 ,  74.08182207],
       [  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,  60.65306597]])

In [4]:
T = 5
dt = 1
m = int(T/dt)
S0 = 100
r = 0.03  # Assume a constant risk-free rate
sigma = 0.10
q = 0

u = np.exp(sigma * np.sqrt(dt))  # up movement size
d = 1 / u  # down movement size
p = (np.exp((r - q) * dt) - d) / (u - d)  # risk-neutral up probability

price_tree = np.zeros((m + 1, m + 1))
option_values = np.zeros((m + 1, m + 1))

for j in range(m + 1):
    for i in range(j + 1):
        price_tree[i, j] = S0 * (u ** (j - i)) * (d ** i)

In [5]:
def black_scholes(sigma, K, r, S0, T, is_call=True, q=0):
    """
    Black-Scholes option pricing
    :param sigma: volatility of the underlying (decimal)
    :param K: strike price
    :param r: risk-free rate (decimal)
    :param S0: initial asset price
    :param T: time until expiration (years)
    :param is_call: True for call, False for put
    :param q: dividend yield (decimal)

    :return: price of option
    """
    d1 = (math.log(S0 / K) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * math.sqrt(T))
    d2 = d1 - sigma * math.sqrt(T)

    if is_call:
        price = S0 * norm.cdf(d1) * math.exp(-q * T) - K * math.exp(-r * T) * norm.cdf(d2)
        return float(price)
    else:
        price = K * math.exp(-r * T) * norm.cdf(-d2) - S0 * norm.cdf(-d1) * math.exp(-q * T)
        return float(price)

In [6]:
price_tree = np.zeros((m + 1, m + 1))
price_tree[0,0] = S0
for i in range(1,m+1):
    n = i + 1  # Number of nodes
    if n % 2 == 0:
        bs_price = black_scholes(sigma,S0,r,S0,dt,is_call=True)
        up_center = (S0 * np.exp(r * dt) * bs_price)/()
        down_center = 0
         
    else:
        center = int(i/2)
        price_tree[center,1] = S0
        

In [7]:
np.floor(2.5)

2.0

New stuff

In [8]:
T = 5
dt = 1
m = int(T/dt)
S0 = 100
r = 0.03  # Assume a constant risk-free rate
sigma = 0.10
q = 0

u = np.exp(sigma * np.sqrt(dt))  # up movement size
d = 1 / u  # down movement size
p = (np.exp((r - q) * dt) - d) / (u - d)  # risk-neutral up probability

price_tree = np.zeros((m + 1, m + 1))
option_values = np.zeros((m + 1, m + 1))
implied_tree = np.zeros((m + 1, m + 1))
ab_tree = np.zeros((m + 1, m + 1))
p_tree = np.zeros((m + 1, m + 1))

for j in range(m + 1):
    for i in range(j + 1):
        price_tree[i, j] = S0 * (u ** (j - i)) * (d ** i)

In [9]:
price_tree

array([[100.        , 110.51709181, 122.14027582, 134.98588076,
        149.18246976, 164.87212707],
       [  0.        ,  90.4837418 , 100.        , 110.51709181,
        122.14027582, 134.98588076],
       [  0.        ,   0.        ,  81.87307531,  90.4837418 ,
        100.        , 110.51709181],
       [  0.        ,   0.        ,   0.        ,  74.08182207,
         81.87307531,  90.4837418 ],
       [  0.        ,   0.        ,   0.        ,   0.        ,
         67.0320046 ,  74.08182207],
       [  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,  60.65306597]])

In [10]:
np.exp(-0.03) * 10.52 * 0.627 

6.401097557107302

In [11]:
is_call = True
is_euro = True
K = S0
m = 5
if is_call:
    option_values[:, m] = np.maximum(np.zeros(m + 1), price_tree[:, m] - K)
else:
    option_values[:, m] = np.maximum(np.zeros(m + 1), K - price_tree[:, m])

for j in range(m - 1, -1, -1):
    for i in range(j + 1):
        cont = np.exp(-r * dt) * (p * option_values[i, j + 1] + (1 - p) * option_values[i + 1, j + 1])
        if is_euro or is_call:
            option_values[i, j] = cont
        else:
            option_values[i, j] = max(K - price_tree[i, j], cont)

option_values

array([[17.19753194, 23.36804993, 31.19835461, 40.8094274 , 52.13791641,
        64.87212707],
       [ 0.        ,  8.22761927, 12.11147819, 17.58725556, 25.09572246,
        34.98588076],
       [ 0.        ,  0.        ,  2.36970825,  3.89429166,  6.39973616,
        10.51709181],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ]])

In [12]:
implied_tree[0,0] = S0
ab_tree[0,0] = 1.0
# p_tree = [0,0]

In [13]:
# implied tree starts with S0, then we have an even number of nodes and need to find S_{i+1} from the central node

normal_u = np.exp(sigma * np.sqrt(dt))
normal_d = 1 / normal_u
normal_p = (np.exp(r * dt) - normal_d) / (normal_u - normal_d)


call = np.exp(-r) * normal_p * (price_tree[0,1]- 100) 
summ = 0 

central_up = S0 * (np.exp(r * dt) * call + ab_tree[0,0] * S0 - summ) / (ab_tree[0,0] * S0 * np.exp(r * dt) - np.exp(r * dt) * call + summ)
central_down = (S0**2) / central_up

In [14]:
call

6.3997361625294085

In [15]:
central_up

110.51709180756477

In [16]:
(103 - central_down) / (central_up - central_down)

0.6247711038805017

# All together hopefully

In [24]:
# Initialize parameters & trees
T = 5
dt = 1
m = int(T/dt)
S0 = 100
r = 0.03  # Assume a constant risk-free rate
sigma = 0.10
skew = 0.0005  # Increase in vol for decrease in strike

price_tree = np.zeros((m + 1, m + 1))
# option_values = np.zeros((m + 1, m + 1))

implied_tree = np.zeros((m + 1, m + 1))
implied_tree[0,0] = S0

ad_tree = np.zeros((m + 1, m + 1))  # Arrow-Debreu price tree
ad_tree[0,0] = 1.0

vol_tree = np.zeros((m + 1, m + 1)) 
vol_tree[0,0] = sigma

p_tree = np.zeros((m + 1, m + 1))  # risk-neutral probability tree

for j in range(m + 1):
    for i in range(j + 1):
        price_tree[i, j] = S0 * (u ** (j - i)) * (d ** i)
        vol_tree[i, j] = sigma - (price_tree[i, j] - S0) * skew

In [32]:
vol_tree

array([[0.1       , 0.09474145, 0.08892986, 0.08250706, 0.07540877,
        0.06756394],
       [0.        , 0.10475813, 0.1       , 0.09474145, 0.08892986,
        0.08250706],
       [0.        , 0.        , 0.10906346, 0.10475813, 0.1       ,
        0.09474145],
       [0.        , 0.        , 0.        , 0.11295909, 0.10906346,
        0.10475813],
       [0.        , 0.        , 0.        , 0.        , 0.116484  ,
        0.11295909],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.11967347]])

In [31]:
price_tree

array([[100.        , 110.51709181, 122.14027582, 134.98588076,
        149.18246976, 164.87212707],
       [  0.        ,  90.4837418 , 100.        , 110.51709181,
        122.14027582, 134.98588076],
       [  0.        ,   0.        ,  81.87307531,  90.4837418 ,
        100.        , 110.51709181],
       [  0.        ,   0.        ,   0.        ,  74.08182207,
         81.87307531,  90.4837418 ],
       [  0.        ,   0.        ,   0.        ,   0.        ,
         67.0320046 ,  74.08182207],
       [  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,  60.65306597]])

In [25]:
# Make sure to update p tree and ad tree
# Change to work for any dt

normal_u = np.exp(sigma * np.sqrt(dt))  # Normal risk-neutral factors/p
normal_d = 1 / normal_u
normal_p = (np.exp(r * dt) - normal_d) / (normal_u - normal_d)

for i in range(2):
    n = i + 2  # Number of nodes
    even_nodes = (n % 2 == 0) 
    
    # Central node(s)
    if even_nodes:
        # Start with central two nodes
        call = np.exp(-r) * normal_p * (price_tree[int(n/2)-1,n-1] - price_tree[int(n/2)-1,n-2]) 
        summ = 0  # FIX THIS - ONLY TRUE FOR N == 2 MIGHT BE FIXED
        price = price_tree[int(n/2)-1,n-2]
        for j in range(int(n/2)-2,-1,-1):
            summ += ad_tree[j,n-2] * ((1+r)*price_tree[j,n-2] - price)
        
        implied_tree[int(n/2)-1, n-1] = S0 * (np.exp(r * dt) * call + ad_tree[int(n/2)-1,n-2] * S0 - summ) / (ad_tree[int(n/2)-1,n-2] * S0 * np.exp(r * dt) - np.exp(r * dt) * call + summ)
        implied_tree[int(n/2),n-1] = (S0**2) / implied_tree[int(n/2)-1, n-1]
        
        first_upper = int(n/2)-2
    else:
        # Middle node equal to spot
        implied_tree[int(np.floor(n/2)),n-1] = S0
        first_upper = int(np.floor(n/2)) - 1
        
    # Upper Nodes   
    for j in range(first_upper,-1,-1):
        implied_tree[j,n-1] = 
        # DOWN ONE implied_tree[j+1,n-1]
        
    # Lower nodes
    
    # probability tree
    for j in range(n-1):
        p_tree[j,n-2] = ((1+r) * price_tree[j,n-2] - implied_tree[j+1,n-1]) / (implied_tree[j,n-1] - implied_tree[j+1,n-1])
        
    # AD Tree
    for j in range(n):
        if j == 0:
            ad_tree[j,n-1] = ad_tree[j,n-2] * p_tree[j,n-2] / (1+r)
        elif j == n-1:
            ad_tree[j,n-1] = ad_tree[j-1,n-2] * (1-p_tree[j-1,n-2]) / (1+r)
        else:
            ad_tree[j,n-1] = (ad_tree[j,n-2] * p_tree[j,n-2] + ad_tree[j-1,n-2] * (1-p_tree[j-1,n-2])) / (1+r)

In [26]:
implied_tree

array([[100.        , 110.51709181,   0.        ,   0.        ,
          0.        ,   0.        ],
       [  0.        ,  90.4837418 , 100.        ,   0.        ,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ]])

In [29]:
p_tree

array([[ 0.6247711 , -0.13832605,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.93198254,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ]])

In [35]:
vol_tree

array([[0.1       , 0.09474145, 0.08892986, 0.08250706, 0.07540877,
        0.06756394],
       [0.        , 0.10475813, 0.1       , 0.09474145, 0.08892986,
        0.08250706],
       [0.        , 0.        , 0.10906346, 0.10475813, 0.1       ,
        0.09474145],
       [0.        , 0.        , 0.        , 0.11295909, 0.10906346,
        0.10475813],
       [0.        , 0.        , 0.        , 0.        , 0.116484  ,
        0.11295909],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.11967347]])

In [61]:
binomial_price(0.094741, 110.517092, 0.03, 100, 2, 2, is_call=True, is_euro=True, q=0)

3.9510547019920144

In [70]:
binomial_price(0.0886, 120.51, 0.03, 120.51, 1, 1, is_call=True, is_euro=True, q=0)

7.037069735885942